In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from tqdm import tqdm
import time
from transformers import AdamW, get_scheduler
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import h3
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer, DataCollatorWithPadding
import torch
from torch import nn
import os
import matplotlib.pyplot as plt

In [2]:
address_data_path = '/home/uddeshya.singh/DataSet_triplet/classification_dataset/PUNJAB_with_grid_ids_train.csv'

In [3]:
df_train = pd.read_csv(address_data_path)

In [4]:
fine_tuned_lm_trained_save_dir = '/home/uddeshya.singh/Models_test/Fine-tuned-save'
fine_tuned_lm_trained_out_dir = '/home/uddeshya.singh/Models_test/Fine-tuned-out'
model_dir = '/home/uddeshya.singh/Experiments/Model-Roberta-lm/Punjab_pan_india_train_threshold_save_dir'

In [5]:
pretrained_tokenizer = AutoTokenizer.from_pretrained(model_dir)

In [6]:
def compute_grid_ids(row, res):
    return h3.geo_to_h3(row['lookup_lat'], row['lookup_lng'], res)

df_train['grid_id_res9'] = df_train.apply(compute_grid_ids, axis=1, res=9)
df_train['grid_id_res10'] = df_train.apply(compute_grid_ids, axis=1, res=10)
df_train['grid_id_res11'] = df_train.apply(compute_grid_ids, axis=1, res=11)

address_data_train = Dataset.from_pandas(df_train)

In [7]:
def clean_dataset(batch):
    if pd.isnull(batch['cleaned_address']) or batch['cleaned_address'] == '':
        return False
    return True
print('starting now ', time.strftime('%X %x %Z'))
address_data_train = address_data_train.filter(clean_dataset)
print('completed now ', time.strftime('%X %x %Z'))

starting now  18:27:25 07/27/24 IST


Filter:   0%|          | 0/6913138 [00:00<?, ? examples/s]

completed now  18:28:23 07/27/24 IST


In [8]:
def tokenize(batch):
    # Ensure that each entry in batch["cleaned_address"] is a string
    assert all(isinstance(x, str) for x in batch["cleaned_address"]), "All inputs must be strings"
    return pretrained_tokenizer(batch["cleaned_address"], padding=True, truncation=True, max_length=100)

address_data_train = address_data_train.remove_columns(['addr_hash', 'lookup_lat', 'lookup_lng' , 'grid_id'])
address_data_train = address_data_train.class_encode_column("grid_id_res10")
address_data_train = address_data_train.class_encode_column("grid_id_res9")
address_data_train = address_data_train.class_encode_column("grid_id_res11")

Flattening the indices:   0%|          | 0/6913118 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/6913118 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/6913118 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/6913118 [00:00<?, ? examples/s]

In [9]:
# import sys
# import pickle
# from pympler import asizeof

In [10]:
train_dataset = address_data_train.map(tokenize, batched=True, batch_size=100000, num_proc=1)

train_dataset = train_dataset.remove_columns(['cleaned_address'])

Map:   0%|          | 0/6913118 [00:00<?, ? examples/s]

In [11]:
# dataset_file_path = "/home/uddeshya.singh/Dataset-ML/train_dataset2.pkl"
# with open(dataset_file_path, 'rb') as f:
#     loaded_train_dataset = pickle.load(f)

In [12]:
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "grid_id_res10", "grid_id_res9", "grid_id_res11"])


In [13]:
def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels_res10 = torch.tensor([item['grid_id_res10'] for item in batch])
    labels_res9 = torch.tensor([item['grid_id_res9'] for item in batch])
    labels_res11 = torch.tensor([item['grid_id_res11'] for item in batch])
    
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=pretrained_tokenizer.pad_token_id)
    attention_mask_padded = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    
    return {
        'input_ids': input_ids_padded,
        'attention_mask': attention_mask_padded,
        'labels_res10': labels_res10,
        'labels_res9': labels_res9,
        'labels_res11': labels_res11
    }

In [14]:
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

In [15]:
for idx,ele in enumerate(train_loader):
    print(idx,ele)
    break

0 {'input_ids': tensor([[    0, 15337,  3117,  6637,   288,     2,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [    0,  9520,   323,  9547,   574,  7088,   333, 31499,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1

In [16]:
num_labels_res10 = address_data_train.features["grid_id_res10"].num_classes
num_labels_res9 = address_data_train.features["grid_id_res9"].num_classes
num_labels_res11 = address_data_train.features["grid_id_res11"].num_classes

In [17]:
class MultiTaskModel(nn.Module):
    def __init__(self, model_name, num_labels_res10, num_labels_res9, num_labels_res11):
        super(MultiTaskModel, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.hidden_size = self.bert.config.hidden_size
        self.classifier_res10 = nn.Linear(self.hidden_size, num_labels_res10)
        self.classifier_res9 = nn.Linear(self.hidden_size, num_labels_res9)
        self.classifier_res11 = nn.Linear(self.hidden_size, num_labels_res11)
    
    def forward(self, input_ids, attention_mask=None, labels_res10=None, labels_res9=None, labels_res11=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]  
        logits_res10 = self.classifier_res10(pooled_output)
        logits_res9 = self.classifier_res9(pooled_output)
        logits_res11 = self.classifier_res11(pooled_output)

        loss = None
        if labels_res10 is not None and labels_res9 is not None and labels_res11 is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss_res10 = loss_fct(logits_res10.view(-1, num_labels_res10), labels_res10.view(-1))
            loss_res9 = loss_fct(logits_res9.view(-1, num_labels_res9), labels_res9.view(-1))
            loss_res11 = loss_fct(logits_res11.view(-1, num_labels_res11), labels_res11.view(-1))
            loss = 0.6 * loss_res10 + 0.3 * loss_res9 + 0.1 * loss_res11

        return (loss, logits_res10, logits_res9, logits_res11)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:" ,device)

Device: cuda


In [19]:
model = MultiTaskModel(model_dir, num_labels_res10, num_labels_res9, num_labels_res11).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
num_epochs = 10

Some weights of RobertaModel were not initialized from the model checkpoint at /home/uddeshya.singh/Experiments/Model-Roberta-lm/Punjab_pan_india_train_threshold_save_dir and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
def train_model(model, train_loader, optimizer, num_epochs, save_dir, out_dir):
    loss_values = []
    save_interval = 20000
    iteration = 0


    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(out_dir, exist_ok=True)
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader):
            input_ids = batch['input_ids'].to("cuda")
            attention_mask = batch['attention_mask'].to("cuda")
            labels_res10 = batch['labels_res10'].to("cuda")
            labels_res9 = batch['labels_res9'].to("cuda")
            labels_res11 = batch['labels_res11'].to("cuda")

            optimizer.zero_grad()
            loss, logits_res10, logits_res9, logits_res11 = model(input_ids, attention_mask, labels_res10, labels_res9, labels_res11)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            iteration += 1
            
            if iteration % save_interval == 0:
                model_save_path = f"{save_dir}/model_MM_checkpoint_{iteration}.pt"
                optimizer_save_path = f"{out_dir}/optimizer_MM_checkpoint_{iteration}.pt"
                torch.save(model.state_dict(), model_save_path)
                torch.save(optimizer.state_dict(), optimizer_save_path)
        
        avg_loss = total_loss / len(train_loader)
        loss_values.append(avg_loss)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}")


    final_model_save_path = f"{save_dir}/MM_model.pt"
    torch.save(model.state_dict(), final_model_save_path)


    optimizer_save_path = f"{out_dir}/optimizer_MMl.pt"
    torch.save(optimizer.state_dict(), optimizer_save_path)


    plt.figure()
    plt.plot(range(1, num_epochs + 1), loss_values, marker='o', label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Over Epochs')
    plt.legend()
    plt.savefig(f"{save_dir}/training_loss.png")
    plt.show()

    return loss_values



print('starting now ', time.strftime('%X %x %Z'))
start_time = time.time()
train_model(model, train_loader, optimizer, num_epochs, fine_tuned_lm_trained_save_dir, fine_tuned_lm_trained_out_dir)
end_time = time.time()
print('completed now ', time.strftime('%X %x %Z'))

starting now  18:43:08 07/27/24 IST


  2%|██▉                                                                                                                                                                                            | 53196/3456559 [3:59:01<254:52:47,  3.71it/s]


KeyboardInterrupt: 

In [ ]:
!free --giga


In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi